# Dataset Prepration
Each image must have a prefix and a suffix. For fine-tuning Florence-2 on an object detection task, the prefix (prompt) is always the same: <OD>. 

The suffix, the expected model response, Each bounding box is described by a string with the following structure: 

{class_name}<loc{x1}><loc{y1}><loc{x2}><loc{y2}>. Here, the values x1, y1, x2, y2  describe the coordinates of the bounding box vertices.  


In [2]:
import os
import json

annotations_dir = "custom_dataset/instruments_yolov8/train/labels"
output_json_file = "custom_dataset/instruments_yolov8/train/images/train_annotations.json"


# annotations_dir = "custom_dataset/instruments_yolov8/valid/labels"
# output_json_file = "custom_dataset/instruments_yolov8/valid/images/val_annotations.json"


def parse_yolo_annotation(annotation_file):
    with open(annotation_file, 'r') as f:
        lines = f.readlines()
   
    image_name = os.path.basename(annotation_file).replace('.txt', '.jpg')
    prefix = "<OD>"
    suffix_lines = []
   
    for line in lines:
        parts = line.strip().split()
        class_name = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
       
        x1 = int((x_center - width/2) * 1000)
        y1 = int((y_center - height/2) * 1000)
        x2 = int((x_center + width/2) * 1000)
        y2 = int((y_center + height/2) * 1000)
       
        # Replace '0' with 'face' in the class name if it equals '0'
        if class_name == '0':
            class_name = 'DC Power Supply'
        elif class_name == '1':
            class_name = 'Digital Multimeter'
        elif class_name == '2':
            class_name = 'Digital Trainer'            
        elif class_name == '3':
            class_name = 'Function Generator'
        elif class_name == '4':
            class_name = 'Oscilloscope'           
        suffix_line = f"{class_name}<loc_{x1}><loc_{y1}><loc_{x2}><loc_{y2}>"
        suffix_lines.append(suffix_line)
   
    json_obj = {
        "image": image_name,
        "prefix": prefix,
        "suffix": "".join(suffix_lines)
    }
   
    return json_obj

annotations_json_strings = []

for filename in os.listdir(annotations_dir):
    if filename.endswith(".txt"):
        annotation_file = os.path.join(annotations_dir, filename)
        annotation_obj = parse_yolo_annotation(annotation_file)
        json_string = json.dumps(annotation_obj, separators=(',', ':'))
        annotations_json_strings.append(json_string)

with open(output_json_file, 'w') as json_file:
    json_file.write("\n".join(annotations_json_strings))

print(f"Annotations have been written to {output_json_file}")


Annotations have been written to custom_dataset/instruments_yolov8/valid/images/val_annotations.json
